In [ ]:
!pip install fastapi uvicorn yfinance nest_asyncio pyngrok --quiet

from fastapi import FastAPI, HTTPException
import yfinance as yf
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn
import pandas as pd
import asyncio

# 🔑 Mets ici ton authtoken ngrok (copie-le depuis https://dashboard.ngrok.com/get-started/your-authtoken)
conf.get_default().auth_token = "348k9xobGsbjedye1zkYDKjhlc5_7kVCF53MZBfHrbrh9Bvxj"

# --- Création de l'application ---
app = FastAPI(
    title="Yahoo Finance API",
    description="API pour récupérer les prix d'ouverture d'une action entre deux dates via Yahoo Finance",
    version="1.3.0"
)

@app.get("/open_prices/")
def get_open_prices(symbol: str, start: str, end: str):
    """
    Récupère les prix d'ouverture quotidiens entre deux dates.
    Exemple : /open_prices?symbol=AAPL&start=2024-01-01&end=2024-02-01
    """
    try:
        data = yf.download(symbol, start=start, end=end)
        if data.empty:
            raise HTTPException(status_code=404, detail=f"Aucune donnée trouvée pour {symbol} entre {start} et {end}.")
        results = [
            {"date": str(date.date()), "open_price": round(float(row["Open"]), 2)}
            for date, row in data.iterrows()
        ]
        return {
            "symbol": symbol.upper(),
            "start_date": start,
            "end_date": end,
            "open_prices": results
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# --- Démarrage correct du serveur dans Colab ---
nest_asyncio.apply()

# Ouvre un tunnel public via ngrok
public_url = ngrok.connect(8000)
print(f"🚀 Ton API est disponible ici : {public_url.public_url}/open_prices?symbol=AAPL&start=2024-01-01&end=2024-02-01")

# Démarre uvicorn de manière asynchrone (compatible Colab)
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

await server.serve()  # ✅ cette ligne fonctionne dans Colab sans RuntimeError

<frozen posixpath>:82: RuntimeWarning: coroutine 'Server.serve' was never awaited


🚀 Ton API est disponible ici : https://persons-pretechnically-beau.ngrok-free.dev/open_prices?symbol=AAPL&start=2024-01-01&end=2024-02-01


INFO:     Started server process [622]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     193.54.195.236:0 - "GET /open_prices?symbol=AAPL&start=2024-01-01&end=2024-02-01 HTTP/1.1" 307 Temporary Redirect


/tmp/ipython-input-1092823193.py:28: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start, end=end)
[*********************100%***********************]  1 of 1 completed

INFO:     193.54.195.236:0 - "GET /open_prices/?symbol=AAPL&start=2024-01-01&end=2024-02-01 HTTP/1.1" 200 OK



/tmp/ipython-input-1092823193.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  {"date": str(date.date()), "open_price": round(float(row["Open"]), 2)}
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [622]


In [ ]:
!pip install yfinance gradio pandas --quiet

import yfinance as yf
import gradio as gr
import pandas as pd
import json
from datetime import date
from google.colab import files

# --- Liste des symboles CAC40 (Yahoo Finance) ---
cac40_symbols = {
    "Air Liquide": "AI.PA",
    "Airbus": "AIR.PA",
    "ArcelorMittal": "MT.AS",
    "AXA": "CS.PA",
    "BNP Paribas": "BNP.PA",
    "Bouygues": "EN.PA",
    "Capgemini": "CAP.PA",
    "Carrefour": "CA.PA",
    "Crédit Agricole": "ACA.PA",
    "Danone": "BN.PA",
    "Dassault Systèmes": "DSY.PA",
    "Engie": "ENGI.PA",
    "EssilorLuxottica": "EL.PA",
    "Eurofins Scientific": "ERF.PA",
    "Hermès": "RMS.PA",
    "Kering": "KER.PA",
    "Legrand": "LR.PA",
    "L'Oréal": "OR.PA",
    "LVMH": "MC.PA",
    "Michelin": "ML.PA",
    "Orange": "ORA.PA",
    "Pernod Ricard": "RI.PA",
    "Renault": "RNO.PA",
    "Safran": "SAF.PA",
    "Saint-Gobain": "SGO.PA",
    "Sanofi": "SAN.PA",
    "Schneider Electric": "SU.PA",
    "Société Générale": "GLE.PA",
    "STMicroelectronics": "STM.PA",
    "Teleperformance": "TEP.PA",
    "Thales": "HO.PA",
    "TotalEnergies": "TTE.PA",
    "Unibail-Rodamco-Westfield": "URW.AS",
    "Veolia": "VIE.PA",
    "Vinci": "DG.PA",
    "Vivendi": "VIV.PA"
}

# --- Fonction pour récupérer les prix d'ouverture et créer JSON ---
def get_open_prices_json_download(stock_name, start, end):
    symbol = cac40_symbols.get(stock_name)
    if not symbol:
        return {"error": "Action non trouvée"}

    try:
        data = yf.download(symbol, start=start, end=end)
        if data.empty:
            return {"error": "Aucune donnée trouvée pour cette période"}

        results = [{"date": str(date.date()), "open_price": round(float(row["Open"]), 2)}
                   for date, row in data.iterrows()]

        json_result = {
            "symbol": stock_name,
            "start_date": start,
            "end_date": end,
            "open_prices": results
        }

        # Enregistrer le JSON dans un fichier
        filename = f"{stock_name}_open_prices.json"
        with open(filename, "w") as f:
            json.dump(json_result, f, indent=4)

        # Préparer le téléchargement
        files.download(filename)

        return json_result

    except Exception as e:
        return {"error": str(e)}

# --- Interface Gradio ---
interface = gr.Interface(
    fn=get_open_prices_json_download,
    inputs=[
        gr.Dropdown(label="Choisissez une action du CAC40", choices=list(cac40_symbols.keys()), value="LVMH"),
        gr.Textbox(label="Date de début (AAAA-MM-JJ)", value="2024-01-01"),
        gr.Textbox(label="Date de fin (AAAA-MM-JJ)", value="2024-01-10")
    ],
    outputs=gr.JSON(label="Prix d'ouverture JSON"),
    title="💹 Prix d'ouverture - Actions du CAC40 (JSON + téléchargement)",
    description="Sélectionnez une action et une période pour obtenir les prix d'ouverture quotidiens et télécharger le JSON."
)

# Lancer l'interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67a7f0b12058d9db3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
import yfinance as yf
import pandas as pd
import sqlite3
from datetime import datetime

# Liste des symboles du CAC40 (sans doublons)
cac40_symbols = [
    "AI.PA", "AIR.PA", "MT.AS", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA",
    "CA.PA", "ACA.PA", "BN.PA", "DSY.PA", "ENGI.PA", "EL.PA", "ERF.PA",
    "RMS.PA", "KER.PA", "LR.PA", "OR.PA", "MC.PA", "ML.PA", "ORA.PA",
    "RI.PA", "RNO.PA", "SAF.PA", "SGO.PA", "STLAM.MI", "STMPA.PA", "SU.PA",
    "GLE.PA", "VIE.PA", "VIV.PA", "HO.PA", "SAN.PA", "STM.PA", "DG.PA", "UG.PA"
]

# Supprimer les doublons éventuels
cac40_symbols = list(set(cac40_symbols))

# Connexion à la base SQLite (création si inexistante)
conn = sqlite3.connect("cac40_open_prices.db")
cursor = conn.cursor()

# Création de la table avec PRIMARY KEY(date, symbol)
cursor.execute("""
CREATE TABLE IF NOT EXISTS cac40_open_prices (
    date TEXT,
    symbol TEXT,
    open_price REAL,
    PRIMARY KEY(date, symbol)
)
""")
conn.commit()

# Définir la période
start_date = datetime(2020, 1, 1)
end_date = datetime.today()

# Boucle pour récupérer et insérer les données
for symbol in cac40_symbols:
    print(f"Récupération de {symbol} ...")
    try:
        df = yf.download(symbol, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"), auto_adjust=True)

        if df.empty:
            print(f"  Pas de données pour {symbol} (peut-être délisté).")
            continue

        df.reset_index(inplace=True)
        df_to_insert = df[['Date', 'Open']].copy()
        df_to_insert['Symbol'] = symbol
        df_to_insert.columns = ['date', 'open_price', 'symbol']

        # Insérer en ignorant les doublons (SQLite PRIMARY KEY)
        df_to_insert.to_sql('cac40_open_prices', conn, if_exists='append', index=False)
        print(f"  {len(df_to_insert)} lignes ajoutées.")

    except Exception as e:
        print(f"  Erreur pour {symbol}: {e}")

# Fermer la connexion
conn.close()
print("Terminé !")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Récupération de STLAM.MI ...
  1475 lignes ajoutées.
Récupération de AI.PA ...
  1485 lignes ajoutées.
Récupération de ML.PA ...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

  1485 lignes ajoutées.
Récupération de SU.PA ...
  1485 lignes ajoutées.
Récupération de HO.PA ...
  1485 lignes ajoutées.
Récupération de ACA.PA ...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...

  1485 lignes ajoutées.
Récupération de EN.PA ...
  1485 lignes ajoutées.
Récupération de DG.PA ...



[*********************100%***********************]  1 of 1 completed


  1485 lignes ajoutées.
Récupération de DSY.PA ...
  1485 lignes ajoutées.
Récupération de EL.PA ...


[*********************100%***********************]  1 of 1 completed

  1485 lignes ajoutées.
Terminé !


In [22]:
import sqlite3
import pandas as pd

# --- Paramètres ---
db_path = "/content/cac40_open_prices.db"  # chemin de ta base
symbol_to_check = "MC.PA"                  # ticker de l'action, ex: LVMH
start_date = "2025-10-12"                  # date de début
end_date = "2025-10-15"                    # date de fin

# --- Connexion à la base ---
conn = sqlite3.connect(db_path)

# --- Requête SQL ---
query = f"""
SELECT date, open_price
FROM cac40_open_prices
WHERE symbol = '{symbol_to_check}'
  AND date BETWEEN '{start_date}' AND '{end_date}'
ORDER BY date ASC
"""

# --- Exécution et affichage ---
df = pd.read_sql(query, conn)
print(df)

# --- Fermer la connexion ---
conn.close()

                  date  open_price
0  2025-10-13 00:00:00  548.200012
1  2025-10-14 00:00:00  534.000000
